In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import os
import pandas as pd
import xlrd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Creates a list of municipalities to extract data from map.

In [200]:
def municipios_func():
    municipios = pd.read_csv('/Users/samielzengotita/Documents/IRONHACK/PROYECT/Real_Estate/PR_Municipios')
    municipios = municipios.values.tolist()
    flat_list = []
    for row in municipios:
        flat_list.extend(row)
    flat_list.remove('Moca')
    #flat_list.remove('Toa Baja')
    muni_pr = []
    for municipio in flat_list:
        muni_pr.append(municipio + ', PR')
    return muni_pr

# Webscraping - Gathering data from map website

In [201]:
def download_data(municipio):
    
    site_input = driver.find_element(By.CSS_SELECTOR,'#searchField')
    site_input.clear()
    site_input.send_keys(municipio)
    button = driver.find_element(By.CSS_SELECTOR,'#searchButton')
    button.click()
    time.sleep(2)
    i = 1
    while True:
        cdbg = driver.find_element(By.CSS_SELECTOR,f'#listOfGranteesFound > option:nth-child({i})')
        if 'CDBG' not in cdbg.text:
            i+=1
            time.sleep(2)
        else:
            break
    cdbg.click()
    finish = driver.find_element(By.CSS_SELECTOR,'#idChooseGranteeFinishBtn')
    finish.click()
    time.sleep(2)
    export = driver.find_element(By.CSS_SELECTOR,'#exportBtn')
    export.click()
    

# Cleans excel files to create the first dataframe, which is CDBG data.

In [216]:
def clean_excel_df1(file):
    df_excel_1 = pd.read_excel((file), skiprows=[0,1,2,3,4,5,6,7])
    df_excel_1.columns = df_excel_1.columns.fillna('type')
    df_excel_1.rename(columns={"type": "2018"}, inplace=True)
    df_excel_1.rename(columns={"nan.1": "2019"}, inplace=True)
    df_excel_1.rename(columns={"nan.2": "2020"}, inplace=True)
    df_excel_1.rename(columns={"nan.3": "2021"}, inplace=True)
    
    df_excel_1 = df_excel_1.drop('Unnamed: 1', axis=1)
    df_excel_1 = df_excel_1.drop('Unnamed: 2', axis=1)
    cleaned_excel_cdbg = df_excel_1.iloc[:1,:]
    return cleaned_excel_cdbg

# Cleans excel files to create the first dataframe, which is the grantees data.

In [229]:
def clean_excel_df2(file):
    df_excel_2 = pd.read_excel((file), skiprows=[0,1,2,3,4,5,6,7])
    df_excel_2.columns = df_excel_2.columns.fillna('type')
    df_excel_2.rename(columns={"type": "2018"}, inplace=True)
    df_excel_2.rename(columns={"nan.1": "2019"}, inplace=True)
    df_excel_2.rename(columns={"nan.2": "2020"}, inplace=True)
    df_excel_2.rename(columns={"nan.3": "2021"}, inplace=True)
    
    df_excel_2 = df_excel_2.drop('Unnamed: 1', axis=1)
    df_excel_2 = df_excel_2.drop('Unnamed: 2', axis=1)
    cleaned_excel_grantee = df_excel_2.iloc[7:28,:]
    
    cleaned_excel_grantee.reset_index(drop=True, inplace=True)
    cleaned_excel_grantee = df2.drop(cleaned_excel_grantee.loc[cleaned_excel_grantee.index==4].index)
    cleaned_excel_grantee = df2.drop(cleaned_excel_grantee.loc[cleaned_excel_grantee.index==5].index)
    cleaned_excel_grantee = df2.drop(cleaned_excel_grantee.loc[cleaned_excel_grantee.index==11].index)
    cleaned_excel_grantee = df2.drop(cleaned_excel_grantee.loc[cleaned_excel_grantee.index==12].index)
    cleaned_excel_grantee = df2.drop(cleaned_excel_grantee.loc[cleaned_excel_grantee.index==13].index)
    cleaned_excel_grantee = df2.drop(cleaned_excel_grantee.loc[cleaned_excel_grantee.index==14].index)
    cleaned_excel_grantee = df2.drop(cleaned_excel_grantee.loc[cleaned_excel_grantee.index==15].index)
    cleaned_excel_grantee.reset_index(drop=True, inplace=True)
    cleaned_excel_grantee.rename(columns={2022: "Amount"}, inplace=True)
    return cleaned_excel_grantee

In [224]:
def clean_excel_df2(file):
    df_excel_2 = pd.read_excel((file), skiprows=[0,1,2,3,4,5,6,7])
    df_excel_2.columns = df_excel_2.columns.fillna('type')
    df_excel_2.rename(columns={"type": "2018"}, inplace=True)
    df_excel_2.rename(columns={"nan.1": "2019"}, inplace=True)
    df_excel_2.rename(columns={"nan.2": "2020"}, inplace=True)
    df_excel_2.rename(columns={"nan.3": "2021"}, inplace=True)
    
    df_excel_2 = df_excel_2.drop('Unnamed: 1', axis=1)
    df_excel_2 = df_excel_2.drop('Unnamed: 2', axis=1)
    cleaned_excel_grantee = df_excel_2.iloc[7:28,:]
    
    cleaned_excel_grantee.reset_index(drop=True, inplace=True)
    cleaned_excel_grantee = df2.drop('2018', axis=1)
    cleaned_excel_grantee = df2.drop('2019', axis=1)
    cleaned_excel_grantee = df2.drop('2020', axis=1)
    cleaned_excel_grantee = df2.drop('2021', axis=1)
    cleaned_excel_grantee = df2.drop(cleaned_excel_grantee.loc[cleaned_excel_grantee.index==4].index)
    cleaned_excel_grantee = df2.drop(cleaned_excel_grantee.loc[cleaned_excel_grantee.index==5].index)
    cleaned_excel_grantee = df2.drop(cleaned_excel_grantee.loc[cleaned_excel_grantee.index==11].index)
    cleaned_excel_grantee = df2.drop(cleaned_excel_grantee.loc[cleaned_excel_grantee.index==12].index)
    cleaned_excel_grantee = df2.drop(cleaned_excel_grantee.loc[cleaned_excel_grantee.index==13].index)
    cleaned_excel_grantee = df2.drop(cleaned_excel_grantee.loc[cleaned_excel_grantee.index==14].index)
    cleaned_excel_grantee = df2.drop(cleaned_excel_grantee.loc[cleaned_excel_grantee.index==15].index)
    cleaned_excel_grantee.reset_index(drop=True, inplace=True)
    cleaned_excel_grantee.rename(columns={2022: "Amount"}, inplace=True)
    return cleaned_excel_grantee

# Gives a dictionary with 26 datasets for each municipality, with CDBG data

In [221]:
cdbg_dict = {}
def cdbg_data():

    dirname = '/Users/samielzengotita/Documents/IRONHACK/PROYECT/Real_Estate/EXCEL FILES'
    ext = ('.xlsx')
    

    
    for file in os.scandir(dirname):
        if file.path.endswith(ext):
            municipality_name = os.path.splitext(file.name)[0]  # Extracting the municipality name from the file name
            df_cdbg_data = clean_excel_df1(file)
            cdbg_dict[municipality_name] = df_cdbg_data
    
    # Now df_dict contains 26 DataFrames, each with a unique title (municipality name)
    # You can access each DataFrame using df_dict['municipality_name']

# Gives a dictionary with 26 datasets for each municipality, with grantee data

In [225]:
grant_dict = {}
def grant_data():

    dirname = '/Users/samielzengotita/Documents/IRONHACK/PROYECT/Real_Estate/EXCEL FILES'
    ext = ('.xlsx')
    
    for file in os.scandir(dirname):
        if file.path.endswith(ext):
            municipality_name = os.path.splitext(file.name)[0]  # Extracting the municipality name from the file name
            df_grant_data = clean_excel_df2(file)
            grant_dict[municipality_name] = df_grant_data
    
    # Now df_dict contains 26 DataFrames, each with a unique title (municipality name)
    # You can access each DataFrame using df_dict['municipality_name']

# Returns a list where you can copy the municipality to call upon any of the 26 datasets created.

In [206]:
def muni_copy():

    muni_copy_list = []
    for key,value in df_dict.items():
        muni_copy.append(key)
    muni_copy_list

# Webscraping - Gathers mortgage data.

In [207]:
def get_fha_snapshots(): 
    i = 5
    while i == 5:
        for j in range(1,13):
            site_input = driver.find_element(By.CSS_SELECTOR,f'#block-system-main > div > div:nth-child(1) > div > div > div > div > div > div > div > div > div.genlink > ul:nth-child({i}) > li:nth-child({j}) > a')
            site_input.click()
            time.sleep(2)
        i = 7
    else:
        for j in range(1,49):
            site_input = driver.find_element(By.CSS_SELECTOR,f'#block-system-main > div > div:nth-child(1) > div > div > div > div > div > div > div > div > div.genlink > ul:nth-child({i}) > li:nth-child({j}) > a')
            site_input.click()
            time.sleep(2)

# Cleans names of mortgage data excel files.

In [199]:
def rename_fha_snapshots():
    
    dirname = '/Users/samielzengotita/Documents/IRONHACK/PROYECT/Real_Estate/FHA SNAPSHOTS'
    
    for file in os.scandir(dirname):
        if file.name.startswith('FHA_SFSnapshot_'):
            os.rename(f'{file.path}', f"{file.path.replace('FHA_SFSnapshot_','')}")
            
    for file in os.scandir(dirname):
        if file.name.startswith('FHA_SFSnapshot Forward_'):
            os.rename(f'{file.path}', f"{file.path.replace('FHA_SFSnapshot Forward_','')}")
            
    for file in os.scandir(dirname):
        if file.name.startswith('FHA_SFSnapshotForward_'):
            os.rename(f'{file.path}', f"{file.path.replace('FHA_SFSnapshotForward_','')}")
            
    for file in os.scandir(dirname):
        if file.name.startswith('FHA_Snapshot_'):
            os.rename(f'{file.path}', f"{file.path.replace('FHA_Snapshot_','')}")
            
    for file in os.scandir(dirname):
        if file.name.startswith('FHASFSnapshotForward_'):
            os.rename(f'{file.path}', f"{file.path.replace('FHASFSnapshotForward_','')}")
            
    for file in os.scandir(dirname):
        if file.name.startswith('FHASFSnapshot_'):
            os.rename(f'{file.path}', f"{file.path.replace('FHASFSnapshot_','')}")
    
    for file in os.scandir(dirname):
        if file.name.endswith('_New.xlsx'):
            os.rename(f'{file.path}', f"{file.path.replace('_New','')}")
    
    for file in os.scandir(dirname):
        if file.name.endswith('..xlsx'):
            os.rename(f'{file.path}', f"{file.path.replace('..xlsx','.xlsx')}")

# Returns interest rate average for each year (2018-2022) and average for mortgage amount.

In [170]:
interest_rate_avg = {}
original_mortgage_amount_avg = {}

def mortgage_avg():

    directory_path = '/Users/samielzengotita/Documents/IRONHACK/PROYECT/Real_Estate/FHA SNAPSHOTS'
    
    
    # Loop through each file in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".xlsx"):  
            
            year = filename[-9:-5]
            
            
            df = pd.read_excel(os.path.join(directory_path, filename), sheet_name=2)  
            
            
            df['Interest Rate'] = pd.to_numeric(df['Interest Rate'], errors='coerce')
            df['Original Mortgage Amount'] = pd.to_numeric(df['Original Mortgage Amount'], errors='coerce')
            
            
            df = df.iloc[1:]
            
            
            if year not in interest_rate_avg:
                interest_rate_avg[year] = []
            interest_rate_avg[year].append(df['Interest Rate'].mean())
            
            if year not in original_mortgage_amount_avg:
                original_mortgage_amount_avg[year] = []
            original_mortgage_amount_avg[year].append(df['Original Mortgage Amount'].mean())
    
    
    interest_rate_df = pd.DataFrame([(year, sum(avg_list) / len(avg_list)) for year, avg_list in interest_rate_avg.items()], columns=['Year', 'Interest Rate Average'])
    original_mortgage_amount_df = pd.DataFrame([(year, sum(avg_list) / len(avg_list)) for year, avg_list in original_mortgage_amount_avg.items()], columns=['Year', 'Original Mortgage Amount Average'])
    
    
    print(interest_rate_df)
    print(original_mortgage_amount_df)

   Year  Interest Rate Average
0  2018               4.761655
1  2020               3.428575
2  2022               4.813071
3  2021               3.106356
4  2019               4.469297
   Year  Original Mortgage Amount Average
0  2018                     208301.660673
1  2020                     233863.245727
2  2022                     279608.412571
3  2021                     253369.659493
4  2019                     217821.891821


# AUTOMATING CODE <3

In [62]:
# options = webdriver.ChromeOptions()
# prefs = {
#     "download.default_directory": "/Users/samielzengotita/Documents/IRONHACK/PROYECT/Real_Estate/EXCEL FILES",
#     "download.directory_upgrade": True,
#     "download.prompt_for_download": False,
# }
# options.add_experimental_option("prefs", prefs)
# 
# driver = webdriver.Chrome(options=options)
# driver.get('https://egis.hud.gov/cpdmaps/')
#     
# list(map(download_data, municipios_func()))

KeyError: "['2018'] not found in axis"

In [234]:
cdbg_data()
grant_data()

In [121]:
# options = webdriver.ChromeOptions()
# prefs = {
#     "download.default_directory": '/Users/samielzengotita/Documents/IRONHACK/PROYECT/Real_Estate/FHA SNAPSHOTS',
#     "download.directory_upgrade": True,
#     "download.prompt_for_download": False,
# }
# options.add_experimental_option("prefs", prefs)
# 
# driver = webdriver.Chrome(options=options)
# driver.get('https://www.hud.gov/program_offices/housing/rmra/oe/rpts/sfsnap/sfsnap')
#     
# get_fha_snapshots()

In [ ]:
rename_fha_snapshots()

In [171]:
interest_rate_df

,Year,Interest Rate Average
0,2018,4.761655
1,2020,3.428575
2,2022,4.813071
3,2021,3.106356
4,2019,4.469297


In [172]:
original_mortgage_amount_df

,Year,Original Mortgage Amount Average
0,2018,208301.660673
1,2020,233863.245727
2,2022,279608.412571
3,2021,253369.659493
4,2019,217821.891821


In [173]:
sorted_interest = interest_rate_df.sort_values(by='Year')
sorted_interest

,Year,Interest Rate Average
0,2018,4.761655
4,2019,4.469297
1,2020,3.428575
3,2021,3.106356
2,2022,4.813071


In [174]:
sorted_mortgage = original_mortgage_amount_df.sort_values(by='Year')
sorted_mortgage

,Year,Original Mortgage Amount Average
0,2018,208301.660673
4,2019,217821.891821
1,2020,233863.245727
3,2021,253369.659493
2,2022,279608.412571
